In [150]:
import numpy as np
import pandas as pd
import math
import os.path
import json
import time

In [151]:
DATA_ROOT_PATH = '/src/data/wi-voting'
WI_COUNTY_TURNOUT = 'WI-county.json'
WI_COUNTY_REG_VOTER = 'RegisteredVotersByCounty_11-01-2020.csv'
COUNTY_FISP = 'county_fips_master.csv'

In [152]:
reg_voter_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_REG_VOTER), encoding='us-ascii')

In [153]:
reg_voter_df['Registered Voters'] = reg_voter_df['Registered Voters'].apply(int)
reg_voter_df=reg_voter_df.rename(columns={'County':'COUNTY'})
reg_voter_df.head(3)

,CountyCode,COUNTY,Registered Voters
0,1,ADAMS COUNTY,12805
1,2,ASHLAND COUNTY,9770
2,3,BARRON COUNTY,27632


In [154]:
reg_voter_df.dtypes

CountyCode            int64
COUNTY               object
Registered Voters     int64
dtype: object

In [155]:
# file -i county_fips_master.csv 
# county_fips_master.csv: application/csv; charset=iso-8859-1

fips_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, COUNTY_FISP), encoding='iso-8859-1')

In [156]:
fips_df.head(3)

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,Autauga County,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,Baldwin County,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,Barbour County,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central


In [157]:
fips_df.dtypes

fips               int64
county_name       object
state_abbr        object
state_name        object
long_name         object
sumlev           float64
region           float64
division         float64
state            float64
county           float64
crosswalk         object
region_name       object
division_name     object
dtype: object

In [158]:
with open(os.path.join(DATA_ROOT_PATH, WI_COUNTY_TURNOUT)) as f:
    wi_county_map = json.load(f)
    turnout_map = wi_county_map['data']
    

In [159]:
print(turnout_map)

{'55001': {'unit': '55001', 'ts': '2020-11-06T00:29:38.960Z', 'pr': 20, 'tp': 20, 'tv': 11806, 'cand': [{'fname': 'Donald', 'name': 'Trump', 'party': 'gop', 'votes': 7362, 'inc': True}, {'fname': 'Joe', 'name': 'Biden', 'party': 'dem', 'votes': 4329, 'winner': 'X'}, {'fname': 'Jo', 'name': 'Jorgensen', 'party': 'lib', 'votes': 85}, {'fname': 'Don', 'name': 'Blankenship', 'party': 'cst', 'votes': 27}, {'fname': 'Brian', 'name': 'Carroll', 'party': 'asp', 'votes': 3}], 'status': 'R'}, '55003': {'unit': '55003', 'ts': '2020-11-06T00:29:38.960Z', 'pr': 17, 'tp': 17, 'tv': 8728, 'cand': [{'fname': 'Joe', 'name': 'Biden', 'party': 'dem', 'votes': 4794, 'winner': 'X'}, {'fname': 'Donald', 'name': 'Trump', 'party': 'gop', 'votes': 3845, 'inc': True}, {'fname': 'Jo', 'name': 'Jorgensen', 'party': 'lib', 'votes': 67}, {'fname': 'Don', 'name': 'Blankenship', 'party': 'cst', 'votes': 11}, {'fname': 'Brian', 'name': 'Carroll', 'party': 'asp', 'votes': 11}], 'status': 'D'}, '55005': {'unit': '55005'

In [160]:
turn_out_per_county = list()
for county_fips, item in turnout_map.items():
    total_vote_count = 0
    vote_dict = dict()
        
    for cand in item['cand']:
        total_vote_count = cand['votes'] + total_vote_count
        if cand['party'] == 'gop':
            vote_dict['REP'] = cand['votes']
        if cand['party'] == 'dem':
            vote_dict['DEM'] = cand['votes']
    if county_fips.isdigit():
        vote_dict['votes'] = total_vote_count
        vote_dict['fips'] = int(county_fips)
        turn_out_per_county.append(vote_dict)

In [161]:
# county FISP -> total votes
print(turn_out_per_county)

[{'REP': 7362, 'DEM': 4329, 'votes': 11806, 'fips': 55001}, {'DEM': 4794, 'REP': 3845, 'votes': 8728, 'fips': 55003}, {'REP': 15803, 'DEM': 9194, 'votes': 25316, 'fips': 55005}, {'DEM': 6155, 'REP': 4617, 'votes': 10876, 'fips': 55007}, {'REP': 75865, 'DEM': 65509, 'votes': 143623, 'fips': 55009}, {'REP': 4834, 'DEM': 2859, 'votes': 7798, 'fips': 55011}, {'REP': 6461, 'DEM': 3569, 'votes': 10138, 'fips': 55013}, {'REP': 18142, 'DEM': 12115, 'votes': 30797, 'fips': 55015}, {'REP': 21316, 'DEM': 14001, 'votes': 35908, 'fips': 55017}, {'REP': 10001, 'DEM': 4520, 'votes': 14866, 'fips': 55019}, {'REP': 16925, 'DEM': 16408, 'votes': 33847, 'fips': 55021}, {'REP': 4620, 'DEM': 3953, 'votes': 8695, 'fips': 55023}, {'DEM': 260157, 'REP': 78789, 'votes': 343599, 'fips': 55025}, {'REP': 31354, 'DEM': 16355, 'votes': 48415, 'fips': 55027}, {'DEM': 10044, 'REP': 9752, 'votes': 20076, 'fips': 55029}, {'DEM': 13214, 'REP': 10919, 'votes': 24595, 'fips': 55031}, {'REP': 13176, 'DEM': 9909, 'votes': 2

In [162]:
turn_out_per_county_df = pd.DataFrame(turn_out_per_county, columns=['fips', 'votes', 'REP', 'DEM'])

In [163]:
turn_out_per_county_df.head(3)

,fips,votes,REP,DEM
0,55001,11806,7362,4329
1,55003,8728,3845,4794
2,55005,25316,15803,9194


In [164]:
turn_out_per_county_df = turn_out_per_county_df.merge(fips_df, on='fips', how='left')

In [165]:
turn_out_per_county_df['COUNTY'] = turn_out_per_county_df['county_name'].str.upper()

In [166]:
turn_out_per_county_df.head(3)

,fips,votes,REP,DEM,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name,COUNTY
0,55001,11806,7362,4329,Adams County,WI,Wisconsin,Adams County WI,50.0,2.0,3.0,55.0,1.0,2-3-55-1,Midwest,East North Central,ADAMS COUNTY
1,55003,8728,3845,4794,Ashland County,WI,Wisconsin,Ashland County WI,50.0,2.0,3.0,55.0,3.0,2-3-55-3,Midwest,East North Central,ASHLAND COUNTY
2,55005,25316,15803,9194,Barron County,WI,Wisconsin,Barron County WI,50.0,2.0,3.0,55.0,5.0,2-3-55-5,Midwest,East North Central,BARRON COUNTY


In [167]:
turn_out_per_county_df = turn_out_per_county_df.merge(reg_voter_df, on='COUNTY', how='left')

In [168]:
turn_out_per_county_df.head(3)

,fips,votes,REP,DEM,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name,COUNTY,CountyCode,Registered Voters
0,55001,11806,7362,4329,Adams County,WI,Wisconsin,Adams County WI,50.0,2.0,3.0,55.0,1.0,2-3-55-1,Midwest,East North Central,ADAMS COUNTY,1,12805
1,55003,8728,3845,4794,Ashland County,WI,Wisconsin,Ashland County WI,50.0,2.0,3.0,55.0,3.0,2-3-55-3,Midwest,East North Central,ASHLAND COUNTY,2,9770
2,55005,25316,15803,9194,Barron County,WI,Wisconsin,Barron County WI,50.0,2.0,3.0,55.0,5.0,2-3-55-5,Midwest,East North Central,BARRON COUNTY,3,27632


In [169]:
turn_out_per_county_df.dtypes

fips                   int64
votes                  int64
REP                    int64
DEM                    int64
county_name           object
state_abbr            object
state_name            object
long_name             object
sumlev               float64
region               float64
division             float64
state                float64
county               float64
crosswalk             object
region_name           object
division_name         object
COUNTY                object
CountyCode             int64
Registered Voters      int64
dtype: object

In [170]:
turn_out_per_county_df['turn_out'] = turn_out_per_county_df['votes'] / turn_out_per_county_df['Registered Voters']

# 2020 Wisconsin Voter Turnout Per County

In [171]:
pd.set_option('display.max_rows', None)
print(turn_out_per_county_df[['COUNTY', 'votes', 'Registered Voters', 'turn_out']].sort_values('turn_out', ascending= False))

                COUNTY   votes  Registered Voters  turn_out
50        PRICE COUNTY    8642               9055  0.954390
60       TAYLOR COUNTY   10678              11294  0.945458
38    MARQUETTE COUNTY    9050               9579  0.944775
28       JUNEAU COUNTY   13683              14567  0.939315
6       BURNETT COUNTY   10138              10795  0.939138
54         RUSK COUNTY    7879               8396  0.938423
69     WAUSHARA COUNTY   13564              14465  0.937712
42       OCONTO COUNTY   23211              24759  0.937477
46        PEPIN COUNTY    4138               4414  0.937472
58      SHAWANO COUNTY   22878              24408  0.937316
30     KEWAUNEE COUNTY   12066              12926  0.933467
7       CALUMET COUNTY   30797              33050  0.931831
66   WASHINGTON COUNTY   88026              94481  0.931679
19  FOND DU LAC COUNTY   57193              61390  0.931634
5       BUFFALO COUNTY    7798               8405  0.927781
32    LAFAYETTE COUNTY    8547          

# 2016 

In [172]:
WI_COUNTY_TURNOUT_2016 = '2016/County by County Report President of the United States Recount.csv'
WI_COUNTY_REG_VOTER_2016 = '2016/registeredvotersbycounty_xlsx_13527.csv'

In [173]:
reg_voter_2016_df = pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_REG_VOTER_2016), encoding='us-ascii')

In [174]:
reg_voter_2016_df.head(3)

,COUNTY,REGISTERED_VOTER
0,ADAMS COUNTY,12476
1,ASHLAND COUNTY,10120
2,BARRON COUNTY,27081


In [175]:
reg_voter_2016_df.dtypes

COUNTY              object
REGISTERED_VOTER     int64
dtype: object

In [176]:
turnout_2016_df = pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_TURNOUT_2016), encoding='us-ascii')

In [177]:
turnout_2016_df.head(3)

,COUNTY,VOTES,REP,DEM
0,ADAMS COUNTY,10130,5966,3745
1,ASHLAND COUNTY,8032,3303,4226
2,BARRON COUNTY,22671,13614,7889


In [178]:
turnout_2016_df.dtypes

COUNTY    object
VOTES      int64
REP        int64
DEM        int64
dtype: object

In [179]:
turn_out_per_county_2016_df = turnout_2016_df.merge(reg_voter_2016_df, on='COUNTY', how='left')

In [180]:
turn_out_per_county_2016_df['TURN_OUT_2016'] = turn_out_per_county_2016_df['VOTES'] / turn_out_per_county_2016_df['REGISTERED_VOTER']

turn_out_per_county_2016_df=turn_out_per_county_2016_df.rename(columns={'VOTES': 'VOTES_2016', 'REGISTERED_VOTER':'REGISTERED_VOTER_2016', 'REP': 'REP_2016', 'DEM': 'DEM_2016'})

# 2016 Wisconsin Voter Turnout Per County

In [181]:
pd.set_option('display.max_rows', None)
print(turn_out_per_county_2016_df[['COUNTY', 'VOTES_2016', 'REGISTERED_VOTER_2016', 'TURN_OUT_2016']].sort_values('TURN_OUT_2016', ascending= False))

                COUNTY  VOTES_2016  REGISTERED_VOTER_2016  TURN_OUT_2016
7       CALUMET COUNTY       26595                  30699       0.866315
54         RUSK COUNTY        7088                   8218       0.862497
8      CHIPPEWA COUNTY       31568                  36651       0.861313
66   WASHINGTON COUNTY       76757                  89277       0.859762
22        GREEN COUNTY       18985                  22117       0.858389
71         WOOD COUNTY       37818                  44072       0.858096
14         DOOR COUNTY       17592                  20527       0.857018
48         POLK COUNTY       22745                  26623       0.854336
19  FOND DU LAC COUNTY       51796                  60664       0.853818
38    MARQUETTE COUNTY        7891                   9256       0.852528
67     WAUKESHA COUNTY      237593                 278896       0.851905
42       OCONTO COUNTY       20206                  23732       0.851424
30     KEWAUNEE COUNTY       10767                 

In [182]:
turn_out_per_county_2020_df = turn_out_per_county_df[['COUNTY', 'votes',  'Registered Voters',  'turn_out', 'REP', 'DEM']].rename(columns={'votes': 'VOTES_2020',  'Registered Voters': 'REGISTERED_VOTER_2020',  'turn_out': 'TURN_OUT_2020', 'REP': 'REP_2020', 'DEM': 'DEM_2020'})

In [183]:
turn_out_per_county_compare_df = turn_out_per_county_2020_df.merge(turn_out_per_county_2016_df, on='COUNTY', how='left')

In [184]:
turn_out_per_county_compare_df['TURN_OUT_DIFF'] = turn_out_per_county_compare_df['TURN_OUT_2020'] - turn_out_per_county_compare_df['TURN_OUT_2016']
turn_out_per_county_compare_df['VOTES_DIFF'] = turn_out_per_county_compare_df['VOTES_2020'] - turn_out_per_county_compare_df['VOTES_2016']
turn_out_per_county_compare_df['REGISTERED_VOTER_DIFF'] = turn_out_per_county_compare_df['REGISTERED_VOTER_2020'] - turn_out_per_county_compare_df['REGISTERED_VOTER_2016']
turn_out_per_county_compare_df['REP_DIFF'] = turn_out_per_county_compare_df['REP_2020'] - turn_out_per_county_compare_df['REP_2016']
turn_out_per_county_compare_df['DEM_DIFF'] = turn_out_per_county_compare_df['DEM_2020'] - turn_out_per_county_compare_df['DEM_2016']

turn_out_per_county_compare_df['DEM_VS_REP_DIFF'] = turn_out_per_county_compare_df['DEM_DIFF'] - turn_out_per_county_compare_df['REP_DIFF']

turn_out_per_county_compare_df['REP_PCT_DIFF'] = turn_out_per_county_compare_df['REP_2020'] / turn_out_per_county_compare_df['REP_2016'] - 1
turn_out_per_county_compare_df['DEM_PCT_DIFF'] = turn_out_per_county_compare_df['DEM_2020'] / turn_out_per_county_compare_df['DEM_2016'] - 1

In [185]:
print(turn_out_per_county_compare_df[['COUNTY', 'TURN_OUT_2016', 'TURN_OUT_2020', 'TURN_OUT_DIFF']].sort_values('TURN_OUT_DIFF', ascending= False))

                COUNTY  TURN_OUT_2016  TURN_OUT_2020  TURN_OUT_DIFF
39    MENOMINEE COUNTY       0.668712       0.885970       0.217258
52     RICHLAND COUNTY       0.777735       0.920851       0.143116
50        PRICE COUNTY       0.814903       0.954390       0.139487
60       TAYLOR COUNTY       0.812055       0.945458       0.133403
11     CRAWFORD COUNTY       0.798182       0.927170       0.128988
25         IRON COUNTY       0.796418       0.913645       0.117227
58      SHAWANO COUNTY       0.820800       0.937316       0.116516
37    MARINETTE COUNTY       0.785960       0.899401       0.113441
0         ADAMS COUNTY       0.811959       0.921984       0.110025
32    LAFAYETTE COUNTY       0.817978       0.927610       0.109632
28       JUNEAU COUNTY       0.830679       0.939315       0.108636
26      JACKSON COUNTY       0.802060       0.903435       0.101375
1       ASHLAND COUNTY       0.793676       0.893347       0.099671
21        GRANT COUNTY       0.815665       0.91

# Compare 2020 vs 2016 bewteen DEM Abs Votes Chg vs REP Abs Votes Chg

In [186]:
print(turn_out_per_county_compare_df[['COUNTY', 'DEM_DIFF', 'REP_DIFF', 'TURN_OUT_DIFF']].sort_values('TURN_OUT_DIFF', ascending= False))

                COUNTY  DEM_DIFF  REP_DIFF  TURN_OUT_DIFF
39    MENOMINEE COUNTY       301        11       0.217258
52     RICHLAND COUNTY       426       864       0.143116
50        PRICE COUNTY       465       835       0.139487
60       TAYLOR COUNTY       300      1077       0.133403
11     CRAWFORD COUNTY       534       784       0.128988
25         IRON COUNTY       258       358       0.117227
58      SHAWANO COUNTY      1063      2678       0.116516
37    MARINETTE COUNTY       954      2181       0.113441
0         ADAMS COUNTY       584      1396       0.110025
32    LAFAYETTE COUNTY       359       843       0.109632
28       JUNEAU COUNTY       674      1619       0.108636
26      JACKSON COUNTY       437       883       0.101375
1       ASHLAND COUNTY       568       542       0.099671
21        GRANT COUNTY       891      1631       0.099379
61  TREMPEALEAU COUNTY       647      1462       0.097732
46        PEPIN COUNTY       145       378       0.095307
69     WAUSHAR

# Compare 2020 vs 2016 bewteen DEM Percentage Chg vs REP Percentage Chg

In [187]:
print(turn_out_per_county_compare_df[['COUNTY', 'DEM_PCT_DIFF', 'REP_PCT_DIFF', 'TURN_OUT_DIFF']].sort_values('TURN_OUT_DIFF', ascending= False))

                COUNTY  DEM_PCT_DIFF  REP_PCT_DIFF  TURN_OUT_DIFF
39    MENOMINEE COUNTY      0.300399      0.041199       0.217258
52     RICHLAND COUNTY      0.119361      0.215300       0.143116
50        PRICE COUNTY      0.174353      0.183154       0.139487
60       TAYLOR COUNTY      0.125366      0.163703       0.133403
11     CRAWFORD COUNTY      0.156186      0.204380       0.128988
25         IRON COUNTY      0.202353      0.172033       0.117227
58      SHAWANO COUNTY      0.175181      0.209727       0.116516
37    MARINETTE COUNTY      0.148853      0.166209       0.113441
0         ADAMS COUNTY      0.155941      0.233993       0.110025
32    LAFAYETTE COUNTY      0.109185      0.211969       0.109632
28       JUNEAU COUNTY      0.165480      0.227069       0.108636
26      JACKSON COUNTY      0.114458      0.179984       0.101375
1       ASHLAND COUNTY      0.134406      0.164093       0.099671
21        GRANT COUNTY      0.088648      0.132065       0.099379
61  TREMPE

In [188]:
print(turn_out_per_county_compare_df[['COUNTY', 'DEM_VS_REP_DIFF', 'TURN_OUT_DIFF']].sort_values('DEM_VS_REP_DIFF', ascending= False))

                COUNTY  DEM_VS_REP_DIFF  TURN_OUT_DIFF
12         DANE COUNTY            34946       0.075638
40    MILWAUKEE COUNTY            20143       0.042783
67     WAUKESHA COUNTY             7553       0.065590
4         BROWN COUNTY             3472       0.040754
31    LA CROSSE COUNTY             3128       0.079051
45      OZAUKEE COUNTY             2897       0.074458
70    WINNEBAGO COUNTY             2663       0.068140
17   EAU CLAIRE COUNTY             2269       0.061430
53         ROCK COUNTY             1670       0.065394
44    OUTAGAMIE COUNTY             1091       0.066948
14         DOOR COUNTY              858       0.062803
56         SAUK COUNTY              723       0.088348
3      BAYFIELD COUNTY              709       0.076703
15      DOUGLAS COUNTY              599       0.085430
43       ONEIDA COUNTY              457       0.068430
57       SAWYER COUNTY              293       0.091471
39    MENOMINEE COUNTY              290       0.217258
22        